In [4]:
import cv2

input_vid = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080822_conditioning/USB camera/hSyn-AS-Gi-1-220808-125748_Cam1.avi"
reference_background = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/example_frame_olena_cleanup.png"
output_vid = input_vid.replace(".avi", "_no_moving_objects.avi")
# Open input video file
cap = cv2.VideoCapture(input_vid)

# Create background subtractor object
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Load reference background image
bg_img = cv2.imread(reference_background)
bg_img = cv2.resize(bg_img, (int(cap.get(3)), int(cap.get(4))))


# Open output video file
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_vid, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Set maximum number of frames to process
n = 12000  # Replace with the desired number of frames to process

# Process each frame in the input video
frame_count = 0
while frame_count < n:
    # Read frame from input video
    ret, frame = cap.read()
    
    if not ret:
        break

    # Apply background subtraction
    mask = bg_subtractor.apply(frame)

    # Apply thresholding to create binary mask
    threshold = 100
    _, binary_mask = cv2.threshold(mask, threshold, 255, cv2.THRESH_BINARY)

    # Apply morphology operations to remove noise and fill holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Invert binary mask to keep non-moving pixels
    binary_mask_inv = cv2.bitwise_not(binary_mask)

    # Apply inverted binary mask to original frame to keep non-moving pixels
    result = cv2.bitwise_and(frame, frame, mask=binary_mask_inv)

    # Fill in the pixels that are being omitted due to the moving object
    result[binary_mask == 255] = bg_img[binary_mask == 255]

    # Write processed frame to output video
    out.write(result)
    
    frame_count += 1

# Release video capture and writer objects
cap.release()
out.release()